In [ ]:
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
class ShowerEnv(Env):
  def __init__(self):
    self.action_space = Discrete(3)
    self.observation_space = Box(low=0, high=100, shape=(1,))
    self.state = 38 + random.randint(-3, 3)
    self.shower_length = 60
  def step(self, action):
    self.state += action - 1
    self.shower_length -= 1

    if self.state >= 37 and self.state <= 39:
      reward = 1
    else:
      reward = -1

    if self.shower_length <= 0:
      done = True
    else:
      done = False
    
    info = {}

    return self.state, reward, done, info

  def render(self):
    pass

  def reset(self):
    self.action_space = Discrete(3)
    self.observation_space = Box(low=0, high=100, shape=(1,))
    self.state = 38 + random.randint(-3, 3)
    self.shower_length = 60
    return self.state

In [ ]:
env = ShowerEnv()
env.observation_space

Box(0.0, 100.0, (1,), float32)

In [ ]:
episodes = 5

for episode in range(1, episodes+1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-56
Episode:2 Score:-28
Episode:3 Score:-22
Episode:4 Score:-50
Episode:5 Score:-22


In [ ]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_13
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -24.8    |
| time/              |          |
|    fps             | 1698     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -28.7       |
| time/                   |             |
|    fps                  | 1245        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007759394 |
|    clip_fraction        | 0.0217      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -0.000367   

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10)

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(59.4, 0.9165151389911681)

In [ ]:
episodes = 5
env = ShowerEnv()
for episode in range(1, episodes+1):
  obs = env.reset()
  done = False
  score = 0

  while not done:
    action, _ = model.predict((obs,))
    obs, reward, done, info = env.step(action)
    score += reward
  print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:40
Episode:2 Score:52
Episode:3 Score:54
Episode:4 Score:46
Episode:5 Score:56


In [ ]:
model.save("shower")